In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Zadanie 3.1

In [ ]:
N = 60
s = cv2.imread('../data/pedestrian/input/in000001.jpg')
XX, YY = s.shape[:2]
BUF = np.zeros((XX, YY, N), np.uint8)
iN = 0

In [ ]:
path, dirs, files = next(os.walk("../data/pedestrian/input/"))
num_samples = len(files)

#### Średnia

In [ ]:
for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    mean = np.mean(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(mean, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    mean = np.mean(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(mean, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

#### Mediana

In [ ]:
for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    median = np.median(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(median, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    median = np.median(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(median, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

## Zadanie 3.2

#### Aproksymacja średniej

In [ ]:
I = cv2.imread('../data/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(np.float64)
alpha = 0.01

for i in range(301, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG = (1 - alpha) * BG + alpha * IG
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

I = cv2.imread('../data/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype(np.float64)
alpha = 0.01

for i in range(301, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG = (1 - alpha) * BG + alpha * IG
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

#### Aproksymacja mediany

In [ ]:
I = cv2.imread('../data/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
alpha = 0.01

for i in range(301, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG[BG < IG] += 1
    BG[BG > IG] -= 1
    # BG[BG == IG] = BG
        
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
I = cv2.imread('../data/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
alpha = 0.01

TP = 0
TN = 0
FP = 0
FN = 0

for i in range(301, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG[BG < IG] += 1
    BG[BG > IG] -= 1
    # BG[BG == IG] = BG
        
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

## Zadanie 3.3

#### Polityka konserwatywna dla aproksymacji średniej

In [ ]:
alpha = 0.01

previous_mask = None

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(BG.astype(np.uint8), IG)
    T, current_mask = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)

    if previous_mask is not None:
        # Aktualizuj model tła tylko tam, gdzie zarówno w poprzedniej, jak i obecnej klatce były piksele tła
        conservative_mask = np.logical_and(previous_mask == 0, current_mask == 0)
        BG[conservative_mask] = (1 - alpha) * BG[conservative_mask] + alpha * IG[conservative_mask]

    previous_mask = current_mask

    median_filtered = cv2.medianBlur(current_mask, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)

cv2.destroyAllWindows()

In [ ]:
alpha = 0.01

previous_mask = None

TP = 0
TN = 0
FP = 0
FN = 0

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(BG.astype(np.uint8), IG)
    T, current_mask = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)

    if previous_mask is not None:
        # Aktualizuj model tła tylko tam, gdzie zarówno w poprzedniej, jak i obecnej klatce były piksele tła
        conservative_mask = np.logical_and(previous_mask == 0, current_mask == 0)
        BG[conservative_mask] = (1 - alpha) * BG[conservative_mask] + alpha * IG[conservative_mask]

    previous_mask = current_mask

    median_filtered = cv2.medianBlur(current_mask, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

## Zadanie 3.4

In [ ]:
MOG = cv2.createBackgroundSubtractorMOG2(history=60, varThreshold=50, detectShadows=True)

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    mask = MOG.apply(I)

    cv2.imshow('diff', mask)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

MOG = cv2.createBackgroundSubtractorMOG2(history=30, varThreshold=20, detectShadows=False)

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    mask = MOG.apply(I)

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((mask > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((mask == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((mask > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((mask == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

## Zadanie 3.5

In [ ]:
KNN = cv2.createBackgroundSubtractorKNN()

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    mask = KNN.apply(I)

    cv2.imshow('diff', mask)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
KNN = cv2.createBackgroundSubtractorKNN(detectShadows=False)

TP = 0
TN = 0
FP = 0
FN = 0

for i in range(300, num_samples):
    I = cv2.imread('../data/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    mask = KNN.apply(I)

    groundtruth = cv2.imread('../data/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((mask > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((mask == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((mask > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((mask == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)